In [1]:
import numpy as np
import pandas as pd

In [2]:
input_file = 'Weighted_paths_0.65_corrected' # this is the 'output_file_1.evaluated.filtered_0.67' file
ranking_file = 'alpha_0.65_corrected.desc'
N = 20 # top ranked genes in the network
output_file = 'subnetwork_0.65_corrected_'+str(N)+'.js'

In [3]:
df = pd.read_csv(input_file,sep='[',skiprows=0,header=None)

In [4]:
df = df.rename(columns={0: 'path_weight', 1: 'genes_path'})

In [5]:
df

,path_weight,genes_path
0,2.3128625134246903e-9-,"'TRIM69','RPS27A','IRAK1']."
1,2.3142540708481324e-9-,"'EPN1','RPS27A','IRAK1']."
2,2.316282487956147e-9-,"'TRIM69','RPS27A','RFC5']."
3,2.3177228295676374e-9-,"'EPN1','RPS27A','RFC5']."
4,2.3475185172454296e-9-,"'LYN','RPS27A','IRAK1']."
...,...,...
11016825,0.0002110144107775081-,"'PIGG','PIGF','PIGO']."
11016826,0.00021828251598526128-,"'CLCNKA','BSND','CLCNKB']."
11016827,0.00024311395395978854-,"'APOB','STAB1','SPARC']."
11016828,0.0002728755664282789-,"'EP300','LCN2','SLC22A17']."


In [6]:
df[['gene1','gene2','gene3']] = df['genes_path'].str.split(',',expand=True)
df[['gene3','brol']] = df['gene3'].str.split(']',expand=True)
df = df.drop(['brol','genes_path'], axis=1)
df[['path_weight','brol']] = df['path_weight'].str.rsplit('-',1,expand=True)
df = df.drop(['brol'], axis=1)
df

/tmp/ipykernel_6424/271992433.py:4: FutureWarning: In a future version of pandas all arguments of StringMethods.rsplit except for the argument 'pat' will be keyword-only.
  df[['path_weight','brol']] = df['path_weight'].str.rsplit('-',1,expand=True)


,path_weight,gene1,gene2,gene3
0,2.3128625134246903e-9,'TRIM69','RPS27A','IRAK1'
1,2.3142540708481324e-9,'EPN1','RPS27A','IRAK1'
2,2.316282487956147e-9,'TRIM69','RPS27A','RFC5'
3,2.3177228295676374e-9,'EPN1','RPS27A','RFC5'
4,2.3475185172454296e-9,'LYN','RPS27A','IRAK1'
...,...,...,...,...
11016825,0.0002110144107775081,'PIGG','PIGF','PIGO'
11016826,0.00021828251598526128,'CLCNKA','BSND','CLCNKB'
11016827,0.00024311395395978854,'APOB','STAB1','SPARC'
11016828,0.0002728755664282789,'EP300','LCN2','SLC22A17'


In [7]:
df_ranking = pd.read_csv(ranking_file,sep='[',skiprows=0,header=None)

In [8]:
df_ranking_top = df_ranking[0:N]
df_ranking_top
top_genes = df_ranking_top[0].values.tolist()
top_genes2 = []
for i in top_genes:
    a = '\''+i+'\''
    top_genes2.append(a)

In [9]:
df_final = df[df['gene1'].isin(top_genes2) | df['gene2'].isin(top_genes2) | df['gene3'].isin(top_genes2)]
df_final

,path_weight,gene1,gene2,gene3
0,2.3128625134246903e-9,'TRIM69','RPS27A','IRAK1'
1,2.3142540708481324e-9,'EPN1','RPS27A','IRAK1'
2,2.316282487956147e-9,'TRIM69','RPS27A','RFC5'
3,2.3177228295676374e-9,'EPN1','RPS27A','RFC5'
4,2.3475185172454296e-9,'LYN','RPS27A','IRAK1'
...,...,...,...,...
11016816,0.0001462547931692978,'SP1','SLC5A1','RSC1A1'
11016822,0.0001944544553377618,'EP300','ARID5B','PHF2'
11016823,0.0002020841367989202,'EP300','OPA1','OMA1'
11016828,0.0002728755664282789,'EP300','LCN2','SLC22A17'


In [10]:
edges = {}
Normalization = sum(df['path_weight'].astype(float)) # Explained in the paper of OMEN
for i in range(N):
    for j in range(N-1-i):
        node1 = top_genes[i]
        node2 = top_genes[i+1+j]
        nodes = []
        nodes.append(top_genes2[i])
        nodes.append(top_genes2[i+1+j])
        df_edge = df_final[ (df_final['gene1'].isin(nodes) & df_final['gene2'].isin(nodes)) | (df_final['gene1'].isin(nodes) & df_final['gene3'].isin(nodes)) | (df_final['gene2'].isin(nodes) & df_final['gene3'].isin(nodes))]
        s = sum(df_edge['path_weight'].astype(float))/Normalization
        if s>0:
            edges[(node1,node2)] = s*100000 # to have bigger values

In [11]:
print(len(edges))
edges

62


{('EP300', 'STAT1'): 5.498173969387239,
 ('EP300', 'STAT2'): 6.4868399478466285,
 ('EP300', 'PRKACA'): 3.0156704239278382,
 ('EP300', 'TP53'): 3.695790640510774,
 ('EP300', 'CTCF'): 4.084368075266448,
 ('EP300', 'MYC'): 3.303647943238049,
 ('EP300', 'CTNNB1'): 2.852898971821856,
 ('EP300', 'SP1'): 3.134445488898977,
 ('EP300', 'RPS27A'): 2.3218248280137024,
 ('EP300', 'PRKACB'): 2.711365356619162,
 ('EP300', 'PRKACG'): 2.722197044558396,
 ('EP300', 'STAT3'): 2.8195629173723082,
 ('EP300', 'EGR1'): 3.451131859301416,
 ('EP300', 'E2F1'): 4.260597478209735,
 ('EP300', 'GPLD1'): 0.05471264991446312,
 ('STAT1', 'STAT2'): 5.045013688498471,
 ('STAT1', 'TP53'): 5.8330376297028605,
 ('STAT1', 'MYC'): 4.752745222008461,
 ('STAT1', 'STAT3'): 2.5276581665543163,
 ('STAT1', 'EGR1'): 4.871334905863068,
 ('STAT1', 'GPLD1'): 0.378178595135485,
 ('STAT2', 'MYC'): 5.214825630589619,
 ('STAT2', 'STAT3'): 5.320403418182538,
 ('PRKACA', 'GNB1'): 2.0856449924134837,
 ('PRKACA', 'CTNNB1'): 2.082810862148722

In [13]:
with open(output_file,'w') as f:
    f.write('graph = {')
    f.write('\n')
    f.write('    nodes: [')
    f.write('\n')
    for i in range(N):
        f.write('        {') 
        f.write('\n')
        f.write('            id: ' + top_genes2[i] + ',') 
        f.write('\n')
        f.write('            rank: ' + str(1+i) + ',') 
        f.write('\n')
        f.write('        },') 
        f.write('\n')
    f.write('    ],')
    f.write('\n')
    f.write('    links: [')
    f.write('\n')
    for edge in edges:
        f.write('        {source: '+ edge[0] +', target: '+edge[1]+', type: "pp", direction: "directed", max_cost: '+ str(edges[edge]) +', evidence: ""},')
        f.write('\n')
    f.write('    ],')
    f.write('\n')
    f.write('}')
f.close()